In [21]:
import torch 
import matplotlib.pyplot as plt 
import cv2
import numpy as np 
import os
from data.manager import load_dataset
from utils.helper import *
from data.helper import get_balanced_sample
from PIL import Image

In [13]:
CONFIG = read_config("/home/julian/indonezia/development/julian/ssl_classification_pipeline/test_config.json")
MODEL_PATH = "/home/julian/indonezia/development/julian/ssl_classification_pipeline/bacil_experiments/run_7/checkpoints/best_model.pth"

In [11]:
train_loader, val_loader, test_loader, input_shape, num_classes,class_names = load_dataset(CONFIG)

Directory already exists at: /home/julian/indonezia/dataset/classification_dataset_julian


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/albumentations/core/validation.py:45: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Class Map (label):  {'bacil': 0, 'debris': 1}
Class Map (idx):  {0: 'bacil', 1: 'debris'}
Class Map (label):  {'bacil': 0, 'debris': 1}
Class Map (idx):  {0: 'bacil', 1: 'debris'}


In [12]:
test_loader_iter = iter(test_loader)
test_images, test_labels = get_balanced_sample(test_loader_iter,n_pos=32,n_neg=32)

In [15]:
from networks.runner import build_model
from utils.helper import get_device
device = get_device()
model = build_model(CONFIG['hyperparams']['network'], CONFIG,input_shape,num_classes)
model = model.to(device)

/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [16]:
def load_pth_model(model, checkpoint_path, device):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()
    return model

In [17]:
model = load_pth_model(model, MODEL_PATH, device)

In [18]:
all_preds_for_test = model(test_images.to(device))  # shape: [B, num_classes]
preds = all_preds_for_test.argmax(dim=1).cpu()
targets = test_labels

In [19]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer=None):
        self.model = model
        self.model.eval()

        # Will store activations and gradients
        self.activations = None
        self.gradients = None

        # Automatically find a convolution layer if none is specified
        if target_layer is None:
            self.target_layer = self.find_last_conv_layer(model)
        else:
            self.target_layer = target_layer

        # Register hooks
        self.hook_forward_backward()

    def find_last_conv_layer(self, model):
        # Reverse through all submodules
        for module in reversed(list(model.modules())):
            if isinstance(module, torch.nn.Conv2d):
                return module
        raise ValueError("No convolutional layer found in the model.")

    def hook_forward_backward(self):
        # Forward hook: to store the *actual* activations of the target layer
        def forward_hook(module, input, output):
            self.activations = output

        # Backward hook: to store the gradients of the target layer
        def backward_hook(module, grad_in, grad_out):
            # grad_out is a tuple, [0] is the actual gradients we need
            self.gradients = grad_out[0]

        # Register forward and backward hooks
        self.target_layer.register_forward_hook(forward_hook)
        # Use register_full_backward_hook if available/preferred (PyTorch 1.10+)
        self.target_layer.register_backward_hook(backward_hook)

    def generate_cam(self, input_image, target_class=None):
        """
        input_image: a 4D tensor [B, C, H, W]
        target_class: integer class index (or None, to use argmax)
        """

        # 1) Forward pass
        output = self.model(input_image)

        # If no target class provided, take the highest-scoring class for the first sample
        if target_class is None:
            # This picks from the *first* item in the batch
            target_class = output.argmax(dim=1).item()

        # 2) Zero grads and backward pass on the target class
        self.model.zero_grad()
        # If your batch size is > 1, make sure you handle that carefully
        loss = output[0, target_class]  # using first sample in batch
        loss.backward()

        # Now self.gradients and self.activations are populated

        # 3) Compute the weights: global-average-pool the gradients
        pooled_gradients = torch.mean(self.gradients, dim=[0, 2, 3])

        # 4) Weight the channels of the stored activations by the pooled gradients
        #    self.activations shape: [B, #channels, H, W]
        #    pooled_gradients shape: [#channels]
        activations = self.activations[0].detach().clone()  # focusing on first sample if batch>1
        for i in range(activations.shape[0]):
            activations[i, :, :] *= pooled_gradients[0]

        # 5) Sum (or mean) across channels to get a single 2D map
        heatmap = torch.mean(activations, dim=0)  # shape: [H, W]

        # 6) ReLU and normalize
        heatmap = F.relu(heatmap)
        if torch.max(heatmap) != 0:
            heatmap /= torch.max(heatmap)

        return heatmap


In [58]:
def create_gradcam_overlay(original_img_np, heatmap_resized_np, alpha=0.6):
    """
    Creates a Grad-CAM overlay using OpenCV.

    Parameters:
    - original_img_np: Original image in [H, W, C] format, values in [0, 1].
    - heatmap_resized_np: Resized heatmap in [H, W], values in [0, 1].
    - alpha: Transparency factor for the heatmap.

    Returns:
    - overlay_bgr: BGR image with the heatmap overlay.
    """
    # Prepare the heatmap
    heatmap_np = np.uint8(255 * heatmap_resized_np)  # Convert to [0, 255] for OpenCV

    # Apply the colormap (e.g., JET)
    heatmap_colored = cv2.applyColorMap(heatmap_np, cv2.COLORMAP_JET)

    # Convert original image to uint8
    original_img_uint8 = np.uint8(255 * original_img_np)

    # Ensure the original image is in BGR format (if it's in RGB)
    original_img_bgr = cv2.cvtColor(original_img_uint8, cv2.COLOR_RGB2BGR)

    # Resize heatmap_colored to match original image size if necessary
    if heatmap_colored.shape[:2] != original_img_bgr.shape[:2]:
        heatmap_colored = cv2.resize(heatmap_colored, (original_img_bgr.shape[1], original_img_bgr.shape[0]))

    # Overlay the heatmap on the original image
    overlay = cv2.addWeighted(heatmap_colored, alpha, original_img_bgr, 1 - alpha, 0)

    return overlay

In [59]:
def save_gradcam_subplot(original_img_np, overlay_bgr, true_label, pred_label, output_path):
    """
    Creates and saves a subplot with the original image and Grad-CAM overlay.

    Parameters:
    - original_img_np: Original image in [H, W, C] format, values in [0, 1].
    - overlay_bgr: BGR image with the heatmap overlay.
    - true_label: String representing the true label.
    - pred_label: String representing the predicted label.
    - output_path: Path to save the subplot image.
    """
    # Convert overlay from BGR to RGB for Matplotlib
    overlay_rgb = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2RGB)

    # Create a Matplotlib figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Original Image
    axs[0].imshow(original_img_np)
    axs[0].axis('off')
    axs[0].set_title(f"Original Image\nLabel: {true_label}", fontsize=12)

    # Grad-CAM Overlay
    axs[1].imshow(overlay_rgb)
    axs[1].axis('off')
    axs[1].set_title(f"Grad-CAM Overlay\nPredicted: {pred_label}", fontsize=12)

    # Adjust layout and save the figure
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free memory


In [61]:
from io import BytesIO

grad_cam = GradCAM(model)

logged_images = []
# Ensure we are not logging an overly large batch; 
# you could trim if needed, e.g. max 16 images
# batch_size = min(images.size(0), 16)
batch_size = test_images.size(0)

# Assuming class_names is a list where class_names[i] corresponds to the label name for class i
# Ensure that test_labels and preds are integer indices
# If they are already indices, proceed. If not, you might need to map them accordingly.

# Define output directory
output_dir = "/home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs"
os.makedirs(output_dir, exist_ok=True)

for idx in range(batch_size):
    input_image = test_images[idx].unsqueeze(0).to(device)

    # Generate the heatmap
    heatmap = grad_cam.generate_cam(input_image)

    # Upsample to the original (H_img, W_img) size using bilinear interpolation
    heatmap_resized = F.interpolate(
        heatmap.unsqueeze(0).unsqueeze(0), 
        size=(test_images.shape[2], test_images.shape[3]),  # (H_img, W_img)
        mode='bilinear', 
        align_corners=False
    )

    # Squeeze back to [H_img, W_img]
    heatmap_resized = heatmap_resized.squeeze(0).squeeze(0)  # shape [H_img, W_img]

    # Normalize the heatmap to [0, 1]
    heatmap_resized = heatmap_resized - heatmap_resized.min()
    if heatmap_resized.max() != 0:
        heatmap_resized = heatmap_resized / heatmap_resized.max()

    # Convert torch tensors to NumPy for OpenCV
    # Original image: (C, H, W) -> (H, W, C)
    original_img_np = test_images[idx].permute(1, 2, 0).cpu().numpy()
    original_img_np = np.clip(original_img_np, 0, 1)  # Ensure values are in [0,1]

    # Create the Grad-CAM overlay using OpenCV
    overlay_bgr = create_gradcam_overlay(original_img_np, heatmap_resized.cpu().numpy(), alpha=0.6)

    # Retrieve true and predicted labels
    true_label_idx = test_labels[idx].item()  # Assuming test_labels are tensor of indices
    pred_label_idx = preds[idx].item()        # Assuming preds are tensor of indices

    true_label = class_names[true_label_idx]
    pred_label = class_names[pred_label_idx]

    # Define the output path for the subplot
    output_path_subplot = os.path.join(output_dir, f"image_{idx}_gradcam_subplot.jpg")

    # Save the subplot with annotations
    save_gradcam_subplot(original_img_np, overlay_bgr, true_label, pred_label, output_path_subplot)

    print(f"Saved Grad-CAM subplot for image {idx} to {output_path_subplot}")


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved Grad-CAM subplot for image 0 to /home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs/image_0_gradcam_subplot.jpg


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved Grad-CAM subplot for image 1 to /home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs/image_1_gradcam_subplot.jpg


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


...

Saved Grad-CAM subplot for image 61 to /home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs/image_61_gradcam_subplot.jpg


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved Grad-CAM subplot for image 62 to /home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs/image_62_gradcam_subplot.jpg


/home/julian/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved Grad-CAM subplot for image 63 to /home/julian/indonezia/development/julian/ssl_classification_pipeline/gradcam_outputs/image_63_gradcam_subplot.jpg
